In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# 1. CARICAMENTO
file_path = 'csv/datadump_s5-000.csv' 
df = pd.read_csv(file_path, low_memory=False)

# 2. FILTRO BOMB
df = df[df['gamemode'] == 'BOMB'].copy()

# 3. SELEZIONE E RINOMINA
# Selezioniamo le colonne necessarie al tuo script
cols_needed = [
    'matchid', 'roundnumber', 'mapname', 'objectivelocation', 
    'role', 'operator', 'nbkills', 'isdead', 'winrole'
]
df_rounds = df[cols_needed].copy()

# Rinominiamo le colonne per farle combaciare con il tuo codice Ridge
# (Il tuo codice usa 'map_name' e 'site_name' nelle righe dell'encoder)
rename_map = {
    'mapname': 'map_name',
    'objectivelocation': 'site_name'
}
df_rounds = df_rounds.rename(columns=rename_map)

# 4. CREAZIONE TARGET 'haswon'
# Il tuo codice prevede la colonna 'haswon'. La creiamo ora.
# Se il ruolo del giocatore è uguale al ruolo vincente, ha vinto (1), altrimenti (0).
df_rounds['haswon'] = (df_rounds['role'] == df_rounds['winrole']).astype(int)

# Pulizia operatori (rimozione prefissi tipo 'SWAT-')
df_rounds['operator'] = df_rounds['operator'].apply(lambda x: x.split('-')[-1] if isinstance(x, str) and '-' in x else x)

# Riempimento eventuali NaN
df_rounds = df_rounds.fillna(0)

print("Dataset pronto per il Ridge!")
print(f"Dimensioni: {df_rounds.shape}")
print("Colonne:", df_rounds.columns.tolist())

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

train, test = train_test_split(df_rounds, test_size=0.2, random_state=11)
#Ridge

features_to_predict = ['haswon']
train_columns = ['matchid', 'roundnumber', 'mapname', 'objectivelocation', 'role', 'operator', 'nbkills', 'isdead']


encoder = LabelEncoder()
df_rounds['map_name'] = encoder.fit_transform(df_rounds['map_name'])
df_rounds['site_name'] = encoder.fit_transform(df_rounds['site_name'])
df_rounds['role'] = encoder.fit_transform(df_rounds['role'])
df_rounds['operator'] = encoder.fit_transform(df_rounds['operator'])
train, test = train_test_split(df_rounds, test_size=0.2, random_state=0)
# Dividing X and Y

X_columns = train_columns
y_column = 'haswon'

X_train = train[X_columns]
y_train = train[y_column]
X_test = test[X_columns]
y_test = test[y_column]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
grid = {'alpha': np.logspace(-6,4,10)}


M = GridSearchCV(estimator  = RidgeClassifier(),
                 param_grid = grid,
                 cv         = 3,
                 scoring    = 'accuracy')
M.fit(X_train, y_train)
y_predict = M.predict(X_test)
acc = accuracy_score(y_test, y_predict)
perr = 100 * (1 - acc)
print(acc)
print(perr)
print(M.best_params_['alpha'])
print('%CM:\n ' +  str(confusion_matrix(y_test,y_predict)))